In [1]:
import pandas as pd
import os
import time
from tqdm import tqdm

# --- CẤU HÌNH ---
# Folder chứa các file CSV hiện tại (sequence dài 601)
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
# Folder để lưu file sau khi xử lý (sequence dài 510)
PREPARED_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data"

# Input hiện tại là 601, Output mong muốn cho SpliceBERT là 510
# (SpliceBERT max 512 tokens = 510 DNA + [CLS] + [SEP])
CURRENT_LEN = 601
TARGET_LEN = 510 

def crop_sequence_worker(sequence):
    """
    Hàm cắt chuỗi từ 601bp xuống 510bp, giữ tâm ở giữa.
    """
    if pd.isna(sequence):
        return "N" * TARGET_LEN
    
    seq_len = len(sequence)
    
    # Nếu chuỗi ngắn hơn target, fill N (trường hợp biên)
    if seq_len < TARGET_LEN:
        return sequence + ("N" * (TARGET_LEN - seq_len))
    
    # Tính toán vị trí cắt để lấy tâm
    # (601 - 510) // 2 = 45. Bắt đầu lấy từ index 45.
    start_idx = (seq_len - TARGET_LEN) // 2
    end_idx = start_idx + TARGET_LEN
    
    # Cắt chuỗi và đảm bảo viết hoa
    return sequence[start_idx:end_idx].upper()

def diagnose_splice_sites(df, sample_size=5):
    """
    Kiểm tra xem sau khi cắt, cặp GT/AG có còn nằm ở giữa (index 255) không.
    """
    print(f"\n{'Type':<10} | {'Window center (idx 253-259)':<30} | {'Found?'}")
    print("-" * 70)
    
    # Với chuỗi 510: tâm cắt (junction) nằm giữa index 254 và 255 (0-based)
    # Thường Donor (GT) sẽ bắt đầu tại index 255
    center_idx = TARGET_LEN // 2  # 255
    
    for label, name in [(1, 'Donor'), (2, 'Acceptor')]:
        samples = df[df['Splicing_types'] == label]
        if len(samples) == 0: continue
        
        test_batch = samples.sample(min(sample_size, len(samples)))
        for _, row in test_batch.iterrows():
            seq = row['sequence']
            
            # Lấy cửa sổ nhỏ quanh tâm mới để kiểm tra
            # Chúng ta kỳ vọng motif nằm ở seq[255:257]
            window = seq[center_idx - 5 : center_idx + 5] 
            
            # Motif hiển thị (đánh dấu cặp nu tại index 255, 256)
            motif = seq[center_idx : center_idx + 2]
            
            target = "GT" if label == 1 else "AG"
            is_match = "✅" if motif == target else "❌"
            
            # Highlight motif trong window để dễ nhìn
            display_win = f"...{seq[center_idx-2:center_idx]} [{motif}] {seq[center_idx+2:center_idx+4]}..."
            
            print(f"{name:<10} | {display_win:<30} | {is_match} (Got: {motif})")

def prepare_csv_datasets(file_list):
    os.makedirs(PREPARED_FOLDER, exist_ok=True)
    
    # Các file cần chạy (khớp với tên file bạn cung cấp ban đầu)
    # Nếu muốn chạy hết folder, dùng os.listdir(DATA_FOLDER)
    if not file_list:
        file_list = [f for f in os.listdir(DATA_FOLDER) if f.endswith('.csv')]

    for file_name in file_list:
        file_start = time.time()
        input_path = os.path.join(DATA_FOLDER, file_name)
        output_path = os.path.join(PREPARED_FOLDER, file_name)
        
        if not os.path.exists(input_path):
            print(f"Skipping {file_name} (Not found)")
            continue
            
        print(f"🚀 Processing: {file_name}...")
        df = pd.read_csv(input_path)
        
        # --- BƯỚC XỬ LÝ CHÍNH ---
        tqdm.pandas(desc="Cropping Sequences")
        df['sequence'] = df['sequence'].progress_apply(crop_sequence_worker)
        
        # Validate độ dài
        df['len_check'] = df['sequence'].apply(len)
        if (df['len_check'] != TARGET_LEN).any():
            print("⚠️ Warning: Có dòng không đủ độ dài 510!")
        df = df.drop(columns=['len_check'])

        # Chẩn đoán (Kiểm tra xem GT/AG có đúng chỗ không)
        diagnose_splice_sites(df)
        
        # Lưu file
        df.to_csv(output_path, index=False)
        
        duration = time.time() - file_start
        print(f"✅ Saved to {output_path} | Speed: {len(df)/duration:.2f} seq/s\n")

# --- CHẠY ---
if __name__ == "__main__":
    # Điền tên các file của bạn vào đây
    my_files = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv']
    prepare_csv_datasets(my_files)

🚀 Processing: test_1_1_1.csv...


Cropping Sequences: 100%|██████████| 26310/26310 [00:00<00:00, 952296.67it/s]


Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------


Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AG...               | ✅ (Got: GT)
Donor      | ...AA [GT] AA...               | ✅ (Got: GT)
Donor      | ...GA [GT] AA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AA...               | ✅ (Got: GT)
Acceptor   | ...TA [GG] AG...               | ❌ (Got: GG)
Acceptor   | ...CA [GG] CG...               | ❌ (Got: GG)
Acceptor   | ...CA [GG] TA...               | ❌ (Got: GG)
Acceptor   | ...TA [GA] CA...               | ❌ (Got: GA)
Acceptor   | ...TA [GG] AA...               | ❌ (Got: GG)
✅ Saved to D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_1_1_1.csv | Speed: 100474.31 seq/s

🚀 Processing: test_2_1_1.csv...


Cropping Sequences: 100%|██████████| 35132/35132 [00:00<00:00, 1063737.87it/s]



Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Donor      | ...TC [GT] GA...               | ✅ (Got: GT)
Donor      | ...TG [GT] AA...               | ✅ (Got: GT)
Donor      | ...AG [GT] CA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AG...               | ✅ (Got: GT)
Acceptor   | ...CA [GA] GA...               | ❌ (Got: GA)
Acceptor   | ...CA [GT] AC...               | ❌ (Got: GT)
Acceptor   | ...CA [GG] GT...               | ❌ (Got: GG)
Acceptor   | ...TA [GA] TT...               | ❌ (Got: GA)
Acceptor   | ...CA [GC] AG...               | ❌ (Got: GC)
✅ Saved to D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_2_1_1.csv | Speed: 130143.47 seq/s

🚀 Processing: test_4_1_1.csv...


Cropping Sequences: 100%|██████████| 52776/52776 [00:00<00:00, 919152.54it/s]



Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...GG [GT] AA...               | ✅ (Got: GT)
Donor      | ...TG [GT] GA...               | ✅ (Got: GT)
Donor      | ...AG [GT] AT...               | ✅ (Got: GT)
Donor      | ...AG [GT] AC...               | ✅ (Got: GT)
Donor      | ...AT [GT] AA...               | ✅ (Got: GT)
Acceptor   | ...CA [GC] AG...               | ❌ (Got: GC)
Acceptor   | ...CA [GG] TT...               | ❌ (Got: GG)
Acceptor   | ...CA [GC] CC...               | ❌ (Got: GC)
Acceptor   | ...CA [GG] AG...               | ❌ (Got: GG)
Acceptor   | ...CA [GG] TG...               | ❌ (Got: GG)
✅ Saved to D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_4_1_1.csv | Speed: 118645.95 seq/s

🚀 Processing: test_10_1_1.csv...


Cropping Sequences: 100%|██████████| 105708/105708 [00:00<00:00, 1071623.80it/s]



Type       | Window center (idx 253-259)    | Found?
----------------------------------------------------------------------
Donor      | ...AG [GT] AG...               | ✅ (Got: GT)
Donor      | ...AA [GT] AA...               | ✅ (Got: GT)
Donor      | ...AG [GT] CA...               | ✅ (Got: GT)
Donor      | ...AG [GT] GA...               | ✅ (Got: GT)
Donor      | ...CG [GT] AA...               | ✅ (Got: GT)
Acceptor   | ...TA [GG] AT...               | ❌ (Got: GG)
Acceptor   | ...CA [GC] AG...               | ❌ (Got: GC)
Acceptor   | ...TA [GG] AA...               | ❌ (Got: GG)
Acceptor   | ...CA [GC] TG...               | ❌ (Got: GC)
Acceptor   | ...CA [GA] GT...               | ❌ (Got: GA)
✅ Saved to D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data\test_10_1_1.csv | Speed: 129275.30 seq/s

